In [ ]:
# How to use zip or tar or rar or tar.gz file in colab

# https://colab.research.google.com/github/sudo-ken/compress-decompress-in-Google-Drive/blob/master/Unrar_Unzip_Rar_Zip_in_GDrive.ipynb#scrollTo=YfSkw7IFH2pl

In [ ]:
# official webpage :- https://machinelearningmastery.com/develop-a-deep-learning-caption-generation-model-in-python/

In [ ]:
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip

In [ ]:
!unzip Flickr8k_Dataset.zip

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip

In [ ]:
!unzip Flickr8k_text.zip

In [ ]:
image_path = '/content/Flicker8k_Dataset'
text_path = '/content/Flickr8k.token.txt'

In [ ]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
device_name = tf.test.gpu_device_name()
print(device_name)

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
with tf.device(device_name):
  # image data set 
  image_id_in_photo=[]
  mapping_image = dict()
  import os
  for name in os.listdir(image_path):
    photo = image_path+'/'+name
    name_id = name.split('.')[0]
    image_id_in_photo.append(name_id)
    if name_id not in mapping_image:
      mapping_image[name_id] = list()
    mapping_image[name_id].append(photo)
  
  # text data set
  text = open(text_path, 'r').read()
  mapping_text = dict()
  image_id_in_text=[]
  for line in text.split('\n'):
    tokens = line.split()
    if len(line) < 2:
      continue
    image_id, image_desc = tokens[0], tokens[1:]
    image_id = image_id.split('.')[0]
    image_desc = ' '.join(image_desc)
    image_id_in_text.append(image_id)
    if image_id not in mapping_text:
      mapping_text[image_id] = list()
    mapping_text[image_id].append(image_desc)

  # check number of data in image_set and text_set
  print('image_id_in_photo --> ',len(image_id_in_photo))
  print('image_id_in_text --> ',len(image_id_in_text),'\n')

  k=''
  for i in image_id_in_text:
    if i not in image_id_in_photo:
      k=str(i)
      print(k)
  temp=[]
  for i in image_id_in_text:
    if str(i) != k:
      temp.append(i)
  image_id_in_text=temp
  mapping_text.pop(k)
  
  print('image_id_in_photo --> ',len(image_id_in_photo))
  print('image_id_in_text --> ',len(image_id_in_text),'\n')

  # set text and photo serially in another folder
  image_set = []
  text_set = []
  for i in image_id_in_photo:
    image_set.append(mapping_image[i])
    text_set.append(mapping_text[i])
  print('image_set --> ',len(image_set))
  print('text_set --> ',len(text_set),'\n')



In [ ]:
with tf.device(device_name):
  # part 1
  from keras.applications.vgg16 import VGG16
  from keras.preprocessing.image import load_img
  from keras.preprocessing.image import img_to_array
  from keras.applications.vgg16 import preprocess_input
  from keras.models import Model

  model = VGG16()
  model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
  #print('\n\n\n',model.summary())

  img_1 = []
  txt_1 = []
  mark = 0
  for i in range(len(image_set)):
    image = load_img(image_set[i][0], target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)

    mark +=1
    if mark == 10:
      break
    
    import string
    table = str.maketrans('', '', string.punctuation)
    p = text_set[i]
    for j in range(len(p)):
      desc = p[j]
      desc = desc.split()
      desc = [word.lower() for word in desc]
      desc = [w.translate(table) for w in desc]
      desc = [word for word in desc if len(word)>1]
      desc = [word for word in desc if word.isalpha()]
      desc = 'startseq ' + ' '.join(desc) + ' endseq'
      img_1.append(feature)
      txt_1.append(desc)

  # part 2
  print('img_1 --> ',len(img_1))
  print('txt_1 --> ',len(txt_1))
  print('DONE!!!')

In [ ]:
# for checking weather the data sets are good fitted or not 
import random
import matplotlib.pyplot as plt
with tf.device(device_name):
  a = random.randint(0,len(image_set)-1)
  image = tf.keras.preprocessing.image.load_img(str(image_set[a][0]),target_size=(224, 224))
  text = text_set[a]
  print(text)
  plt.imshow(image)
  plt.show()

In [ ]:
#img_1 = img_1[:5*100]
#txt_1 = txt_1[:5*100]

In [ ]:
with tf.device(device_name):
  # part 3
  x_1_train = []
  x_2_train = []
  y_train = []
  
  import tensorflow
  from keras.preprocessing.text import Tokenizer, text_to_word_sequence
  from keras.preprocessing.sequence import pad_sequences
  from tensorflow.keras.utils import to_categorical

  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(txt_1)
  text_1 = tokenizer.texts_to_sequences(txt_1)

  maxlen = max([max(i) for i in text_1])

  for j in range(len(text_1)):
    seq = text_1[j]
    ph = img_1[j]
    for i in range(1,len(seq)):
      in_seq , out_seq = seq[:i], seq[i]
      in_seq = pad_sequences([in_seq], maxlen=maxlen+1)[0]
      out_seq = to_categorical([out_seq], num_classes=maxlen+1)[0]
      x_1_train.append(ph)
      x_2_train.append(in_seq)
      y_train.append(out_seq)

  # part 4
  print('x_1_train --> ',len(x_1_train))
  print('x_2_train --> ',len(x_2_train))
  print('y_train --> ',len(y_train))


In [7]:


# part 5
# from tensorflow.keras.utils import plot_model
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import add
# from tensorflow.keras.callbacks import ModelCheckpoint

def model_function(image_size = 4096, text_maxlen=1024):
    inputs1 = Input(shape=(image_size,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    inputs2 = Input(shape=(text_maxlen+1,))
    se1 = Embedding(text_maxlen+1, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256,return_sequences=True)(se2)
    se4 = Dropout(0.5)(se3)
    se5 = LSTM(256)(se4)

    decoder1 = add([fe2, se5])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(text_maxlen+1, activation='softmax')(decoder2)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    return model

model = model_function(image_size = 4096, text_maxlen=1024)
print(model.summary())
#   plot_model(model, to_file='model.png', show_shapes=True)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 1025)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1025, 256)    262400      ['input_7[0][0]']                
                                                                                                  
 dropout_4 (Dropout)            (None, 1025, 256)    0           ['embedding[0][0]']              
                                                                                                  
 input_6 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                              

In [ ]:
import numpy as np
x_1_train = np.array(x_1_train)
x_2_train = np.array(x_2_train)
y_train = np.array(y_train)

print(x_1_train.shape)
print(x_2_train.shape)
print(y_train.shape)

In [ ]:
x_1_train = x_1_train.reshape((x_1_train.shape[0],x_1_train.shape[2]))
print(x_1_train.shape)

In [ ]:
with tf.device(device_name):
  model.fit([x_1_train, x_2_train], y_train, epochs = 10, shuffle = True, batch_size = 256, validation_data =([x_1_train, x_2_train], y_train) )

In [ ]:
model.save('/content/drive/My Drive/model_image_to_text.h5') 

In [ ]:
model.save('/content/drive/My Drive/model_image_to_text_copy.h5')

In [ ]:
'''
with tf.device(device_name):
  filename = '/content/drive/My Drive/DATA_SET/CNN/image_1976.jpg'
  import matplotlib.pyplot as plt
  filename = load_img(filename,target_size=(224, 224))
  plt.imshow(filename)
  plt.axis('off')
  plt.show()
'''

In [ ]:
with tf.device(device_name):
  from numpy import argmax
  from keras.preprocessing.sequence import pad_sequences
  from keras.applications.vgg16 import VGG16
  from keras.preprocessing.image import load_img
  from keras.preprocessing.image import img_to_array
  from keras.applications.vgg16 import preprocess_input
  from keras.models import Model
  from keras.models import load_model

  model_2 = load_model('/content/drive/My Drive/model_image_to_text.h5')

  #text
  #filename = '/content/drive/My Drive/DATA_SET/CNN/image_1976.jpg'
  filename = image_set[5][0]

  model_1 = VGG16()
  model_1 = Model(inputs=model_1.inputs, outputs=model_1.layers[-2].output)
  image = load_img(filename, target_size=(224, 224))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  feature = model_1.predict(image, verbose=0)
	
  in_text = 'startseq'
  for i in range(maxlen+1):
    sequence = tokenizer.texts_to_sequences([in_text])[0]
    sequence = pad_sequences([sequence], maxlen=maxlen+1)
    yhat = model_2.predict([feature,sequence], verbose=0)
    yhat = argmax(yhat)
    word = ''
    for word_1, index in tokenizer.word_index.items():
      if index == yhat:
        word = word_1
    if word is None:
      break
    in_text += ' ' + word
    if word == 'endseq':
      break
  in_text = in_text.split()
  in_text.pop(0)
  in_text.pop(len(in_text)-1)
  in_text = ' '.join(in_text)
  print(in_text)

  #image
  import matplotlib.pyplot as plt
  filename = load_img(filename,target_size=(224, 224))
  plt.imshow(filename)
  plt.axis('off')
  plt.show()

In [ ]:
filename = image_set[5][0]
filename